## 이제 계획한 전체 디렉토리를 기준으로 현재 프로젝트를 재구성해줘

네, 현재 프로젝트를 새로운 구조로 재구성하겠습니다. 먼저 필요한 디렉토리 구조를 생성하겠습니다.


```bash
mkdir -p service-groups/{identity-group/{auth-service,user-service},business-group/{item-service,transaction-service},analytics-group/{reporting-service,notification-service}}/{src/{controllers,models,routes,middleware,config},tests} gateway/src/{routes,middleware,config} gateway/tests client/src/{features/{identity,business,analytics}/{components,hooks,services,types},shared/{components,hooks,services,utils,types,constants},core/{api,auth,router,store},styles/{theme,global,modules},assets/{images,icons,fonts}} client/tests shared/{types,utils,constants} infrastructure/{kubernetes,monitoring,logging}
```
이제 기존 파일들을 새로운 구조로 이동시키겠습니다. 먼저 현재 서버 코드를 identity-group의 auth-service로 이동하겠습니다:


```bash
mv server/index.js service-groups/identity-group/auth-service/src/ && mv server/package.json service-groups/identity-group/auth-service/ && mv server/create_users_table.sql service-groups/identity-group/auth-service/src/config/
```
이제 각 서비스의 기본 package.json 파일을 생성하겠습니다. 먼저 auth-service의 package.json을 업데이트하겠습니다:


```json:service-groups/identity-group/auth-service/package.json
{
  "name": "@todaqinvest/auth-service",
  "version": "1.0.0",
  "description": "Authentication service for Todaqinvest",
  "main": "src/index.js",
  "scripts": {
    "start": "node src/index.js",
    "dev": "nodemon src/index.js",
    "test": "jest"
  },
  "keywords": ["auth", "authentication", "identity"],
  "author": "",
  "license": "ISC",
  "dependencies": {
    "bcrypt": "^5.1.1",
    "cors": "^2.8.5",
    "dotenv": "^16.4.1",
    "express": "^4.18.2",
    "jsonwebtoken": "^9.0.2",
    "pg": "^8.11.3"
  },
  "devDependencies": {
    "nodemon": "^3.0.3",
    "jest": "^29.7.0"
  }
}
```
이제 루트 디렉토리에 docker-compose.yml 파일을 생성하여 전체 서비스를 구성하겠습니다:



```compose:docker-compose.yml
version: '3.8'

services:
  # Identity Group Services
  auth-service:
    build: 
      context: ./service-groups/identity-group/auth-service
      dockerfile: Dockerfile
    ports:
      - "5001:5001"
    environment:
      - NODE_ENV=development
      - DB_USER=${DB_USER}
      - DB_HOST=postgres
      - DB_NAME=${DB_NAME}
      - DB_PASSWORD=${DB_PASSWORD}
      - DB_PORT=5432
      - JWT_SECRET=${JWT_SECRET}
    depends_on:
      - postgres

  user-service:
    build: 
      context: ./service-groups/identity-group/user-service
      dockerfile: Dockerfile
    ports:
      - "5002:5002"
    environment:
      - NODE_ENV=development
      - DB_USER=${DB_USER}
      - DB_HOST=postgres
      - DB_NAME=${DB_NAME}
      - DB_PASSWORD=${DB_PASSWORD}
      - DB_PORT=5432
    depends_on:
      - postgres

  # Business Group Services
  item-service:
    build: 
      context: ./service-groups/business-group/item-service
      dockerfile: Dockerfile
    ports:
      - "5003:5003"
    environment:
      - NODE_ENV=development
      - DB_USER=${DB_USER}
      - DB_HOST=postgres
      - DB_NAME=${DB_NAME}
      - DB_PASSWORD=${DB_PASSWORD}
      - DB_PORT=5432
    depends_on:
      - postgres

  # Analytics Group Services
  reporting-service:
    build: 
      context: ./service-groups/analytics-group/reporting-service
      dockerfile: Dockerfile
    ports:
      - "5004:5004"
    environment:
      - NODE_ENV=development
      - DB_USER=${DB_USER}
      - DB_HOST=postgres
      - DB_NAME=${DB_NAME}
      - DB_PASSWORD=${DB_PASSWORD}
      - DB_PORT=5432
    depends_on:
      - postgres

  # API Gateway
  gateway:
    build: 
      context: ./gateway
      dockerfile: Dockerfile
    ports:
      - "5000:5000"
    environment:
      - NODE_ENV=development
    depends_on:
      - auth-service
      - user-service
      - item-service
      - reporting-service

  # Frontend
  client:
    build: 
      context: ./client
      dockerfile: Dockerfile
    ports:
      - "3000:3000"
    environment:
      - NODE_ENV=development
    depends_on:
      - gateway

  # Database
  postgres:
    image: postgres:14-alpine
    ports:
      - "5432:5432"
    environment:
      - POSTGRES_USER=${DB_USER}
      - POSTGRES_PASSWORD=${DB_PASSWORD}
      - POSTGRES_DB=${DB_NAME}
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  postgres_data:
```
이제 루트 디렉토리에 .env.example 파일을 생성하여 필요한 환경 변수를 정의하겠습니다:



```shell script:.env.example
# Database Configuration
DB_USER=postgres
DB_PASSWORD=your_password
DB_NAME=todaqinvest
DB_HOST=localhost
DB_PORT=5432

# JWT Configuration
JWT_SECRET=your_jwt_secret_key

# Service Ports
AUTH_SERVICE_PORT=5001
USER_SERVICE_PORT=5002
ITEM_SERVICE_PORT=5003
REPORTING_SERVICE_PORT=5004
GATEWAY_PORT=5000
CLIENT_PORT=3000

# Node Environment
NODE_ENV=development
```
마지막으로 README.md 파일을 생성하여 프로젝트 구조와 실행 방법을 문서화하겠습니다:


```markdown:README.md
# Todaqinvest Web Platform

마이크로서비스 아키텍처 기반의 웹 플랫폼

## 프로젝트 구조

```
todaqinvest-web/
├── service-groups/              # 백엔드 서비스 그룹
│   ├── identity-group/         # 인증/사용자 관련 서비스
│   ├── business-group/         # 비즈니스 로직 관련 서비스
│   └── analytics-group/        # 분석 관련 서비스
│
├── client/                     # 프론트엔드 애플리케이션
├── gateway/                    # API 게이트웨이
├── shared/                     # 공유 모듈
└── infrastructure/             # 인프라스트럭처 설정
```

## 시작하기

### 사전 요구사항

- Docker
- Docker Compose
- Node.js 18+
- PostgreSQL 14+

### 환경 설정

1. 환경 변수 설정
```bash
cp .env.example .env
# .env 파일을 수정하여 필요한 환경 변수 설정
```

2. 의존성 설치
```bash
# 각 서비스 디렉토리에서
npm install
```

### 개발 서버 실행

```bash
# 전체 서비스 실행
docker-compose up

# 특정 서비스만 실행
docker-compose up auth-service
docker-compose up user-service
docker-compose up item-service
docker-compose up reporting-service
docker-compose up gateway
docker-compose up client
```

## 서비스 포트

- API Gateway: 5000
- Auth Service: 5001
- User Service: 5002
- Item Service: 5003
- Reporting Service: 5004
- Client: 3000
- PostgreSQL: 5432

## 개발 가이드

### 새로운 서비스 추가

1. service-groups 디렉토리 아래에 새로운 서비스 디렉토리 생성
2. 필요한 의존성 설치
3. docker-compose.yml에 서비스 추가
4. 환경 변수 설정

### API 문서

각 서비스의 API 문서는 해당 서비스의 `/docs` 엔드포인트에서 확인할 수 있습니다.

## 테스트

```bash
# 특정 서비스의 테스트 실행
cd service-groups/identity-group/auth-service
npm test

# 전체 테스트 실행
npm run test:all
```

## 배포

배포 관련 문서는 infrastructure 디렉토리를 참조하세요.
```
이제 기본적인 프로젝트 구조가 재구성되었습니다. 다음 단계로 진행하기 위해서는:

1. 각 서비스별 Dockerfile 생성
2. API Gateway 구현
3. 각 서비스의 기본 코드 구조 구현
4. 프론트엔드 기본 구조 설정

이 중에서 어떤 부분부터 진행하면 좋을지 말씀해 주시면 진행하도록 하겠습니다.

또한, 현재 구조에서 추가하거나 수정하고 싶은 부분이 있다면 말씀해 주세요.
